# Ag-Analytics® - Farmland Sales API 

Ag-Analytics® Farmland Sales API provides users with easy and fast access to filter and find land for sale, complete with the sales and geographic data components to get a detailed description of that land’s value. The data of this service is originally provided by [Farmland Finder](https://www.farmlandfinder.com). In this version of Land Value Service API, each parcel/land has a full description of the entire sale/transaction. Multiple parcel/land record can belong to one transaction. In geographic attribute, each result record gives the detailed description of a certain land/parcel itself, while the attribute in ‘sales’ may carry the information for the transaction/sale other than this parcel/land.

### Required libraries

In [ ]:
import requests
import json
import pandas as pd
import folium
import mplleaflet
import geojson
import shapely.wkt
import os
from urllib.parse import urlencode, quote_plus, quote

### Request Parameter Details

Request URL:  https://ag-analytics.azure-api.net/farmland-sales/

       
1). __Location Parameters__(Required): 
    To make a valid request,at least one location parameters need to be provided.
    
    i. State:  The name of State in string format.Title cased.
    
    ii. County: The name of State in string format. Title cased.This parameter will be valid only if the State parameter is provided
    
    iii.Bounding_box: Area of interest in geoJSON format(See example below).
    

2). __Sale Condition Parameters__(optional): Optional parameters to specify response based on sale conditions

    i. Status: Sale Condition of the property. Vaild options are: Sold, For Sale, Expired Listing.
    
    ii. StartDate (Required only if Status is Sold):Searching starting date of the property Sale Date. In format 'yyyy-mm-dd'
    
    iii.EndDate (Required only if Status is Sold):Searching end date of the property Sale Date. In format 'yyyy-mm-dd'

### Response Description

__Listing Information__

1).Listing_id: The unique ID for each listing(transaction). 

2).Entry_Updated:The date of the sales information has been updated

3).Avg_CSR2: the average The Iowa Corn Suitability Rating(Soil Productivity Index)

4).CRP: If the property joined the Conservation Reserve Program.(‘Yes’ or ‘No’)

5).Total_Acres: the total acres of the entire sale

6).Tillable_Acres: the tillable acres of the sale

7).Percent_Tillable: the percent tillable area

8).CRP_Acres: the CRP acres

9).Sale_Price: the total sale price of the sale record

10).Price_Acre: the price per acre of the sale record

11).Status: one of the following values: "For Sale", "Listing Expired", "Sold"

12).Sale_Condition: one of the following values: "Auction", "Listing"

13).Listing_Agent: the listing agent name

14).Buyer: the buyer name as a string

15).Sale_Date: The sale date string in YYYY-MM-DD format. (When the parcel is still listing, the attribute will be Null

16).Taxes_total: taxes for the sale as a float

17).Assessed_Land: If the land/parcel has been assessed

18).Broker_URL: The URL link to the broker listing webpage as a string


__Parcel Information__

1).Parcel_ID :The unique sale id for each parcel as a string

2).Parcel: Index of the parcel in one listing(transaction).

3).Shape: The boundary of the parcel/property in Well Known Text type

4).GeoJSON: The boundary of the parcel/property in GeoJSON type

5).Acres: Area of the parcel

6).State: The state where the parcel/property locates in. 

7).County:The county where the parcel/property locates in.

8).lat_center: the latitudinal center of the parcel/property as a float value

9).lng_center: the longitudinal center of the parcel/property as a float value

10).range: the range as a string (ex: 26W - always include E or W)

11).sect: the section as string (ex: 17)

12).twnshp: the township as a string (ex: 78N - always include N or S)

13).county_name: the county name as a string (Title cased - for ex: Osceola, Polk, etc.)

14).state_name: the state name as a string (Title cased - for ex: Michigan)

15).STATEFP: the FIPS to state level as a string

16).FIPS: the FIPS to county level as a string


### Request Parameter Example

In [27]:
State= "Missouri"
County= "Audrain"
Bounding_box =  '{ "type": "Feature", "properties": {}, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -88.932982, 46.997269 ], [ -88.932993, 46.99897 ], [ -88.932997, 46.999537 ], [ -88.932997, 46.999636 ], [ -88.932999, 46.999934 ], [ -88.933, 47.000034 ], [ -88.932998, 47.000111 ], [ -88.932994, 47.000343 ], [ -88.932994, 47.000421 ], [ -88.932987, 47.000449 ], [ -88.932967, 47.000535 ], [ -88.932961, 47.000564 ], [ -88.932958, 47.00109 ], [ -88.932951, 47.002671 ], [ -88.932949, 47.003199 ], [ -88.932957, 47.005373 ], [ -88.932982, 47.011395 ], [ -88.93299, 47.011898 ], [ -88.933025, 47.014073 ], [ -88.933046, 47.015422 ], [ -88.933111, 47.019471 ], [ -88.933133, 47.020821 ], [ -88.93314, 47.021299 ], [ -88.933163, 47.022736 ], [ -88.933171, 47.023215 ], [ -88.933183, 47.023969 ], [ -88.933219, 47.026231 ], [ -88.933232, 47.026986 ], [ -88.933246, 47.02784 ], [ -88.933288, 47.030402 ], [ -88.933303, 47.031256 ] ] ] ] } }'
Status = 'For Sale' 
StartDate = '2016-01-01'
EndDate = '2020-06-10'

#Format the inputs()
values = {
    "State":State,
    "County" : County,
    'Bounding_Box': Bounding_box,
    'StartDate':StartDate,
    'EndDate':EndDate,
    "Status":Status
}

# Header for using a subscription key.
headers = {'Content-Type': 'application/x-www-form-urlencoded','Ocp-Apim-Subscription-Key': "XXXXXXXXXXXXXX"}

### API Function with GET request

In [28]:
def LandValueService(values,headers):
    try:
        encode_vars = values.copy()
        if 'Bounding_Box' in values:
            encode_vars.pop('Bounding_Box')
            encoded = urlencode(encode_vars)
            encoded_box = "&Bounding_box=" + "%20" + quote(str(values['Bounding_Box'])) + "'"
            url = encoded + encoded_box
            
        else:
            url = urlencode(values)

        url = "https://ag-analytics.azure-api.net/farmland-sales/?" + url
        response = requests.get(url = url, headers = headers)
        
        #response = requests.get(url, json = values,headers = headers).json()
        print(response.json())

    
    except Exception as e:
        print(e)
        raise e

### API Function with POST request (retired)

Note: As of June 20th, 2020, we have retired the post request for FarmLand Sales Services. Sorry for any inconveniences caused by this!

In [29]:
def LandValueService_Post(values,headers):
    try:
        url = 'https://ag-analytics.azure-api.net/farmland-sales/'
        
        response = requests.post(url, data = values,headers = headers).json()
        print(response.get('status'))
        print(response.get('msg'))
        print(response)
        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

In [ ]:
LandValueResponse = LandValueService(values,headers)

### Format Response

In [ ]:
listing_info =[]
records = LandValueResponse.get('records')
for listing in records:
    parcels_info = records.get(listing).get('parcels')
    list_item = records.get(listing)
    list_item.pop('parcels')
    for parcel in parcels_info:
        parcel.update(list_item)
        listing_info.append(parcel)

listing_db = pd.DataFrame(listing_info)

### Display Response

In [ ]:
#Narrow table
listing_db.set_index('Listing_id').T
#Wide table
# listing_db.set_index('Listing_id')

### Visualize Parcels on Map

In [ ]:
#Display parcels on map
zoom_point=listing_db['GeoJSON'][0]['coordinates'][0][0]
m = folium.Map([zoom_point[1],zoom_point[0]],tiles='Cartodb Positron', zoom_start=11,width='70%', height='100%')
for shape in listing_db['GeoJSON']:
    folium.GeoJson(shape).add_to(m)
m

### Download Response

In [ ]:
download_path = '<---path on your local directory--->' # ex:r'C:\Users\YourName\Downloads'
listing_db.to_csv(os.path.join(download_path,'FarmLand_Listing.csv'))